In [1]:
import pandas as pd
import numpy as np
from EIMTC.preprocessing import OneHotEncoderEIMTC, M1CNNPreprocessing
from EIMTC.selection import train_test_split
from EIMTC.metrics import classification_report
from EIMTC.models import M1CNN

In [2]:
filepath = './data/out.csv'
df = pd.read_csv(filepath, usecols=['udps.n_bytes', 'os', 'browser'])
df

,udps.n_bytes,os,browser
0,"[0.08627450980392157, 0.011764705882352941, 0....",Linux,Firefox
1,"[0.08627450980392157, 0.011764705882352941, 0....",Linux,Chrome
2,"[0.08627450980392157, 0.011764705882352941, 0....",OSX,Safari
3,"[0.08627450980392157, 0.011764705882352941, 0....",Windows,Firefox
4,"[0.08627450980392157, 0.011764705882352941, 0....",Linux,Chrome
...,...,...,...
20628,"[0.08627450980392157, 0.011764705882352941, 0....",Windows,IExplorer
20629,"[0.08627450980392157, 0.011764705882352941, 0....",Windows,IExplorer
20630,"[0.08627450980392157, 0.011764705882352941, 0....",Linux,Firefox
20631,"[0.08627450980392157, 0.011764705882352941, 0....",Linux,Firefox


In [3]:
def preprocessing(df):
    df.dropna(inplace=True)
    M1CNNPreprocessing.preprocess_features(dataframe=df)
    enc = OneHotEncoderEIMTC()
    df['os_ohc'] = list(enc.fit_transform(df['os']).toarray())
    return df

In [4]:
df = preprocessing(df)
df

,udps.n_bytes,os,browser,os_ohc
0,"[0.08627450980392157, 0.011764705882352941, 0....",Linux,Firefox,"[1.0, 0.0, 0.0]"
1,"[0.08627450980392157, 0.011764705882352941, 0....",Linux,Chrome,"[1.0, 0.0, 0.0]"
2,"[0.08627450980392157, 0.011764705882352941, 0....",OSX,Safari,"[0.0, 1.0, 0.0]"
3,"[0.08627450980392157, 0.011764705882352941, 0....",Windows,Firefox,"[0.0, 0.0, 1.0]"
4,"[0.08627450980392157, 0.011764705882352941, 0....",Linux,Chrome,"[1.0, 0.0, 0.0]"
...,...,...,...,...
20628,"[0.08627450980392157, 0.011764705882352941, 0....",Windows,IExplorer,"[0.0, 0.0, 1.0]"
20629,"[0.08627450980392157, 0.011764705882352941, 0....",Windows,IExplorer,"[0.0, 0.0, 1.0]"
20630,"[0.08627450980392157, 0.011764705882352941, 0....",Linux,Firefox,"[1.0, 0.0, 0.0]"
20631,"[0.08627450980392157, 0.011764705882352941, 0....",Linux,Firefox,"[1.0, 0.0, 0.0]"


In [5]:
x_train, x_test, y_train, y_test = train_test_split(df['udps.n_bytes'].values, df['os_ohc'].values, 
    test_size=0.25,
    stratify=df['os'].values,
    random_state=42)

In [6]:
n_bytes = len(df['udps.n_bytes'].iloc[0])
n_classes = len(df['os'].unique())
model = M1CNN(payload_size=n_bytes, n_classes=n_classes)
print(n_bytes, n_classes)

784 3


In [7]:
epochs = 10
batch_size = 128
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)
model.fit(
    np.stack(x_train), 
    np.stack(y_train), 
    epochs=epochs, 
    batch_size=batch_size, 
    use_multiprocessing=True,
    workers=4,
    verbose=2
)

Epoch 1/10
121/121 - 4s - loss: 0.3198 - accuracy: 0.8594
Epoch 2/10
121/121 - 1s - loss: 0.1093 - accuracy: 0.9492
Epoch 3/10
121/121 - 1s - loss: 0.0897 - accuracy: 0.9575
Epoch 4/10
121/121 - 1s - loss: 0.0791 - accuracy: 0.9625
Epoch 5/10
121/121 - 1s - loss: 0.0670 - accuracy: 0.9678
Epoch 6/10
121/121 - 1s - loss: 0.0547 - accuracy: 0.9733
Epoch 7/10
121/121 - 1s - loss: 0.0474 - accuracy: 0.9785
Epoch 8/10
121/121 - 1s - loss: 0.0443 - accuracy: 0.9790
Epoch 9/10
121/121 - 1s - loss: 0.0379 - accuracy: 0.9832
Epoch 10/10
121/121 - 1s - loss: 0.0352 - accuracy: 0.9844


In [8]:
predictions = model.model.predict(np.stack(x_test))
predictions = np.argmax(predictions, axis=1)
y_test_true = np.argmax(np.stack(y_test), axis=1)

report = classification_report(y_test_true, predictions)
report

'              precision    recall  f1-score   support\n\n           0       0.94      0.95      0.95      1410\n           1       1.00      0.98      0.99       667\n           2       0.97      0.97      0.97      3081\n\n    accuracy                           0.97      5158\n   macro avg       0.97      0.97      0.97      5158\nweighted avg       0.97      0.97      0.97      5158\n'

In [9]:
with open('m1cnn_boa2016_os_report.txt', "w+") as f:
    f.write(report)
